In [1]:
import os
import numpy as np
import h5py

In [2]:
file_11 = []
file_12 = []
file_29 = []
file_30 = []
file_39 = []

def walk(path):
    if not os.path.exists(path):
        print("path error")
        return -1
    for root, dirs, filenames in os.walk(path):
        for filename in filenames:
            if str(filename) == '000011.SZtick.mat':
                file_11.append(os.path.join(root,filename))
            elif str(filename) == '000012.SZtick.mat':
                file_12.append(os.path.join(root,filename))
            elif str(filename) == '000029.SZtick.mat':
                file_29.append(os.path.join(root,filename))
            elif str(filename) == '000030.SZtick.mat':
                file_30.append(os.path.join(root,filename))
            elif str(filename) == '000039.SZtick.mat':
                file_39.append(os.path.join(root,filename))
            else:
                print("unknown file name")
                return -1
    return 0

In [3]:
ret = walk(r'data-15.07.08')
if ret == -1:
    assert False

In [9]:
for datapath in file_11:
    print(datapath)
    f = h5py.File(datapath, 'r+')    
    pr = np.array(f['r1']['Price'])
    print(pr.shape)

data-15.07.08/7.6/000011.SZtick.mat
(1, 3479)
data-15.07.08/7.3/000011.SZtick.mat
(1, 3417)
data-15.07.08/8.12/000011.SZtick.mat
(1, 4783)
data-15.07.08/7.21/000011.SZtick.mat
(1, 4535)
data-15.07.08/8.19/000011.SZtick.mat
(1, 4725)
data-15.07.08/7.31/000011.SZtick.mat
(1, 3874)
data-15.07.08/8.31/000011.SZtick.mat
(1, 4601)
data-15.07.08/7.29/000011.SZtick.mat
(1, 4386)
data-15.07.08/7.7/000011.SZtick.mat
(1, 2660)
data-15.07.08/8.21/000011.SZtick.mat
(1, 4588)
data-15.07.08/8.4/000011.SZtick.mat
(1, 4772)
data-15.07.08/8.17/000011.SZtick.mat
(1, 4735)
data-15.07.08/7.20/000011.SZtick.mat
(1, 4485)
data-15.07.08/8.27/000011.SZtick.mat
(1, 4670)
data-15.07.08/8.28/000011.SZtick.mat
(1, 3314)
data-15.07.08/7.22/000011.SZtick.mat
(1, 4610)
data-15.07.08/8.10/000011.SZtick.mat
(1, 4748)
data-15.07.08/7.2/000011.SZtick.mat
(1, 3913)
data-15.07.08/7.8/000011.SZtick.mat
(1, 2369)
data-15.07.08/7.17/000011.SZtick.mat
(1, 4340)
data-15.07.08/7.1/000011.SZtick.mat
(1, 4372)
data-15.07.08/8.6/00

In [4]:
def calc_b(SZtick_filenames):
    b_all = 0
    for dataPath in SZtick_filenames:
        f = h5py.File(dataPath,'r+')
        f['r1'].keys()
        keys = ['AccTurover', 'AccVolume', 'AskAvPrice', 'AskPrice', 'AskVolume', 'BSFlag', 'BidAvPrice', 'BidPrice', 'BidVolume', 'Code', 'CurDelta', 'Date', 'Downs', 'High', 'HoldLines', 'Index', 'Interest', 'Low', 'MatchItems', 'Open', 'Position', 'PreClose', 'PrePosition', 'PreSettle', 'Price', 'Settle', 'Stocks', 'Time', 'TotalAskVolume', 'TotalBidVolume', 'TradeFlag', 'Turover', 'Ups', 'Volume', 'WindCode']
        data = []
        for key in keys:
            data.append(f['r1'][key])
        BidPrice = np.array(f['r1']['BidPrice'])
        AskPrice = np.array(f['r1']['AskPrice'])
        BidVolume = np.array(f['r1']['BidVolume'])
        AskVolume = np.array(f['r1']['AskVolume'])
        TotalBidVolume = np.array(f['r1']['TotalBidVolume'])## 添加于2.12.16.55
        TotalAskVolume = np.array(f['r1']['TotalAskVolume'])
        Price = np.array(f['r1']['Price']) 
        miu_n = BidVolume.astype(int) - AskVolume.astype(int)
        Total_miu_n = TotalBidVolume.astype(int) - TotalAskVolume.astype(int) 

        Price_change = np.zeros(Price.shape)
        Total_miu_n_change = np.zeros(Total_miu_n.shape)
        # Price_change[0,1:Y_Price-1] = Price[0,1:Y_Price-1] - Price[0,0:Y_Price-2]

        [dummy,Y_totmiun] = Total_miu_n.shape
        [dummy,Y_Price] = Price.shape

        DIST_TOTALMIU = 1
        for i in range(DIST_TOTALMIU,Y_totmiun-1): 
            Total_miu_n_change[0,i] = Total_miu_n[0,i] - Total_miu_n[0,i-DIST_TOTALMIU]
        for i in range(1,DIST_TOTALMIU-1):
            Total_miu_n_change[0,i] = Total_miu_n[0,i]

        DIST_PRICE = 5
        for i in range(DIST_PRICE,Y_Price-1): 
            Price_change[0,i] = Price[0,i] - Price[0,i-DIST_PRICE]
            if Price_change[0,i] > 1:  #强行删去一个很偏离的点
                Price_change[0,i] = 0
        for i in range(1,DIST_PRICE-1):
            Price_change[0,i] = Price[0,i]

        Price_change = np.zeros(Price.shape)
        # Price_change[0,1:Y_Price-1] = Price[0,1:Y_Price-1] - Price[0,0:Y_Price-2]
        for i in range(1,Y_Price-1): 
            Price_change[0,i] = Price[0,i] - Price[0,i-1]
            if Price_change[0,i] > 1:  #强行删去一个很偏离的点
                Price_change[0,i] = 0
        Price_change[0,0] = Price[0,0]


        from sklearn.model_selection import train_test_split

        # temp1 =  np.arange(1,Y_totmiun).reshape(1,Y_totmiun-1)#转为1*Y_totmiun-1的矩阵，以使得shape和Total_miu_n匹配
        # temp2 =  np.arange(1,Y_totmiun).reshape(1,Y_totmiun-1)#但这种方式不被train_test_split接受，它要求两个都是数组而不是矩阵
        # X_train,X_test,Y_train,Y_test = train_test_split(temp1[0,0:Y_totmiun-1],temp2[0,0:Y_totmiun-1])

        # X_train,X_test,Y_train,Y_test = train_test_split(Total_miu_n_change[0,0:Y_totmiun-1],Price_change[0,0:Y_Price-1],test_size=0)

        from sklearn.linear_model import LinearRegression

        X_train = Total_miu_n_change[0,0:Y_totmiun-1]
        Y_train = Price_change[0,0:Y_Price-1]
        model=LinearRegression()
        model.fit(X_train.reshape(-1, 1),Y_train.reshape(-1, 1))
        # model.score(X_test.reshape(-1, 1),Y_test.reshape(-1, 1))


        a=model.intercept_#截距
        b=model.coef_#回归系数

        # print(b)

        b_all += b

    b_average = b_all / len(SZtick_filenames)
    return b_average

In [5]:
print(calc_b(file_11))
print(calc_b(file_12))
print(calc_b(file_29))
print(calc_b(file_30))
print(calc_b(file_39))

[[5.43590162e-08]]
[[1.15763041e-08]]
[[2.66610065e-08]]
[[2.80343495e-08]]
[[2.90726565e-08]]
